In [0]:
import os
import tempfile
import time

import numpy as np
import pandas as pd

#Talib used to create ATR indicator
import talib
from logbook import Logger

from catalyst import run_algorithm
from catalyst.api import symbol, record, order_target_percent
from catalyst.exchange.utils.stats_utils import extract_transactions
from catalyst.utils.paths import ensure_directory

NAMESPACE = 'Trailing ATR script'
log = Logger(NAMESPACE)


def initialize(context):
    context.asset = symbol('eth_btc')
    context.base_price = None
    context.current_day = None
    
    # Parameters to tune trailing ATR stop
    context.ATR_period = 5
    context.ATR_multiplier = 3.5
    context.xATR = 0
    
    context.CANDLE_SIZE = '15T'

    context.start_time = time.time()

    context.i = 0


def handle_data(context, data):
    context.i += 1
    if context.i < context.ATR_period:
        return

    today = data.current_dt.floor('1D')
    if today != context.current_day:
        context.traded_today = False
        context.current_day = today
        
    prices = data.history(
        context.asset,
        fields=['high', 'low', 'close'],
        bar_count=20,
        frequency=context.CANDLE_SIZE
    )

    # Ta-lib calculates various technical indicator based on price and
    # volume arrays.

    # Now we are computing ATR bound
    atr = talib.ATR(prices['high'],prices['low'],prices['close'],timeperiod=context.ATR_period)
    nLoss = context.ATR_multiplier * atr[-1]
    

    current = data.current(context.asset, fields=['open', 'high', 'low', 'close', 'volume'])
    price = current['close']

    # Defining trailing ATR stop.
    def xATRTrailingStop():
        if context.i == context.ATR_period:
            context.xATR = price + nLoss
            return context.xATR
        else:
            if price > context.xATR and prices['close'][-2] > context.xATR:
                context.xATR = max(context.xATR, price - nLoss)
                return context.xATR
            elif price < context.xATR and prices['close'][-2] < context.xATR:
                context.xATR = min(context.xATR, price + nLoss)
                return context.xATR
            elif price > context.xATR:
                context.xATR = price - nLoss
                return context.xATR
            else:
                context.xATR = price + nLoss
                return context.xATR
       
    # If base_price is not set, we use the current value. This is the
    # price at the first bar which we reference to calculate price_change.
    if context.base_price is None:
        context.base_price = price

    price_change = (price - context.base_price) / context.base_price
    cash = context.portfolio.cash

    record(
        volume=current['volume'],
        price=price,
        price_change=price_change,
        atr=xATRTrailingStop(),
        cash=cash
    )


    orders = context.blotter.open_orders
    if len(orders) > 0:
        log.info('exiting because orders are open: {}'.format(orders))
        return

    # Exit if we cannot trade
    if not data.can_trade(context.asset):
        return

    pos_amount = context.portfolio.positions[context.asset].amount

    if price > xATRTrailingStop() and pos_amount == 0:
        log.info(
            '{}: buying - price: {}, xATRTrailingStop: {}'.format(
                data.current_dt, price, xATRTrailingStop()
            )
        )
        order_target_percent(context.asset, 1)
        context.traded_today = True

    elif price < xATRTrailingStop() and pos_amount > 0:
        log.info(
            '{}: selling - price: {}, xATRTrailingStop: {}'.format(
                data.current_dt, price, xATRTrailingStop()
            )
        )
        order_target_percent(context.asset, 0)
        context.traded_today = True
    


def analyze(context=None, perf=None):
    end = time.time()
    log.info('elapsed time: {}'.format(end - context.start_time))

    import matplotlib.pyplot as plt
    # The quote currency of the algo exchange
    quote_currency = list(context.exchanges.values())[0].quote_currency.upper()

    # Plot the portfolio value over time.
    ax1 = plt.subplot(611)
    perf.loc[:, 'portfolio_value'].plot(ax=ax1)
    ax1.set_ylabel('Portfolio\nValue\n({})'.format(quote_currency))

   # Plot the price increase or decrease over time.
    ax2 = plt.subplot(612, sharex=ax1)
    perf.loc[:, ['price', 'atr']].plot(ax=ax2, label='Price')

    ax2.set_ylabel('{asset}\n({quote})'.format(
        asset=context.asset, quote=quote_currency
    ))

    transaction_df = extract_transactions(perf)
    if not transaction_df.empty:
        buy_df = transaction_df[transaction_df['amount'] > 0]
        sell_df = transaction_df[transaction_df['amount'] < 0]
        ax2.scatter(
            buy_df.index.to_pydatetime(),
            perf.loc[buy_df.index.floor('1 min'), 'price'],
            marker='^',
            s=100,
            c='green',
            label=''
        )
        ax2.scatter(
            sell_df.index.to_pydatetime(),
            perf.loc[sell_df.index.floor('1 min'), 'price'],
            marker='v',
            s=100,
            c='red',
            label=''
        )

    ax4 = plt.subplot(613, sharex=ax1)
    perf.loc[:, 'cash'].plot(
        ax=ax4, label='Quote Currency ({})'.format(quote_currency)
    )
    ax4.set_ylabel('Cash\n({})'.format(quote_currency))

    perf['algorithm'] = perf.loc[:, 'algorithm_period_return']

    ax5 = plt.subplot(614, sharex=ax1)
    perf.loc[:, ['algorithm', 'price_change']].plot(ax=ax5)
    ax5.set_ylabel('Percent\nChange')


    # Show the plot.
    plt.gcf().set_size_inches(18, 8)
    plt.show()
    pass


if __name__ == '__main__':
    # The execution mode: backtest or live
    live = False

    if live:
        run_algorithm(
            capital_base=0.1,
            initialize=initialize,
            handle_data=handle_data,
            analyze=analyze,
            exchange_name='binance',
            live=True,
            algo_namespace=NAMESPACE,
            quote_currency='eth',
            live_graph=False,
            simulate_orders=False,
            stats_output=None,
            # auth_aliases=dict(poloniex='auth2')
        )

    else:
        folder = os.path.join(
            tempfile.gettempdir(), 'catalyst', NAMESPACE
        )
        ensure_directory(folder)

        timestr = time.strftime('%Y%m%d-%H%M%S')
        out = os.path.join(folder, '{}.p'.format(timestr))
        run_algorithm(
            capital_base=0.035,
            data_frequency='minute',
            initialize=initialize,
            handle_data=handle_data,
            analyze=analyze,
            exchange_name='bitfinex',
            algo_namespace=NAMESPACE,
            quote_currency='btc',
            start=pd.to_datetime('2017-10-01', utc=True).tz_convert('UTC'),
            end=pd.to_datetime('2017-11-01', utc=True).tz_convert('UTC'),
            output=out
        )
        log.info('saved perf stats: {}'.format(out))